In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import dask.dataframe as ddf
from pandas import Series, DataFrame
%matplotlib inline
import seaborn as sn

In [2]:
import geopandas as gpd
import pandas as pd

## Load daily apparent temperature from method 1

In [3]:
apparent_temp = ddf.read_csv('/global/cfs/cdirs/m1532/Projects_MVP/geospatial/climate_heatwave/apparent_temperature/apparent_temperature_method1/apparent_temperature_imputed_method1.csv', dtype = {'fips': str}).compute().drop(columns={'Unnamed: 0'})
apparent_temp.head()

,year,month,date,fips,tMean,tMin,tMax,AT_mean,AT_min,AT_max
0,1960,1,1960-01-01,01001,5.417989,2.635663,8.200316,5.658506,2.748637,8.633515
1,1960,1,1960-01-02,01001,7.005736,4.201073,9.810398,7.347604,4.378348,10.388789
2,1960,1,1960-01-03,01001,9.061477,5.235675,12.887278,9.569066,5.465944,13.821140
3,1960,1,1960-01-04,01001,5.334175,-0.744570,11.412921,5.569946,-0.711797,12.162972
4,1960,1,1960-01-05,01001,6.495222,3.006032,9.984412,6.802082,3.132560,10.580093


In [4]:
city = pd.read_csv('/global/cfs/cdirs/m1532/Projects_MVP/geospatial/climate_heatwave/50_city_county_level.csv', dtype = {'fips': str})

In [5]:
city.head()

,Unnamed: 0,NAME,CLASS,ST,STFIPS,PLACEFIPS,POP2010,STPLFIPS,geometry,fips
0,0,Seattle,city,WA,53,63000,608660,5363000,MULTIPOLYGON (((-13619425.9562 6033333.6701000...,53033
1,1,Portland,city,OR,41,59000,583776,4159000,MULTIPOLYGON (((-13646153.5976 5694643.1055999...,41051
2,2,San Francisco,city,CA,6,67000,805235,667000,MULTIPOLYGON (((-13623645.882399999 4546557.17...,06075
3,3,Fresno,city,CA,6,27000,494665,627000,"MULTIPOLYGON (((-13345406.2071 4398263.0374, -...",06019
4,4,Los Angeles,city,CA,6,44000,3792621,644000,MULTIPOLYGON (((-13162830.3816 3996851.7908999...,06037


In [14]:
climate = city[['NAME', 'fips']].merge(apparent_temp, on = 'fips', how = 'inner')
climate

,NAME,fips,year,month,date,tMean,tMin,tMax,AT_mean,AT_min,AT_max
0,Seattle,53033,1960,1,1960-01-01,-2.959183,-6.680200,0.761834,-2.940829,-6.629042,0.821056
1,Seattle,53033,1960,1,1960-01-02,-3.495462,-5.916871,-1.074053,-3.476559,-5.877712,-1.045205
2,Seattle,53033,1960,1,1960-01-03,-3.105760,-6.547404,0.335883,-3.087406,-6.498515,0.386174
3,Seattle,53033,1960,1,1960-01-04,-2.948310,-5.962373,0.065753,-2.929952,-5.922571,0.110983
4,Seattle,53033,1960,1,1960-01-05,-0.580383,-2.889711,1.728945,-0.545411,-2.871318,1.812890
...,...,...,...,...,...,...,...,...,...,...,...
1095745,Charlotte,37119,2019,12,2019-12-27,9.079570,0.844111,17.315029,9.588802,0.905194,18.969762
1095746,Charlotte,37119,2019,12,2019-12-28,11.610246,4.044080,19.176413,12.383418,4.214061,21.219896
1095747,Charlotte,37119,2019,12,2019-12-29,15.413021,10.411667,20.414375,16.725003,11.051171,22.747983
1095748,Charlotte,37119,2019,12,2019-12-30,16.835078,13.232830,20.437326,18.398317,14.213567,22.776562


In [15]:
climate['NAME'].nunique()

50

In [16]:
def fahrenheit_to_celsius(f):
    c = (f -32)*5/9
    return c

In [17]:
climate['AT_mean'].describe()

count    1.095750e+06
mean     1.620686e+01
std      1.124801e+01
min     -2.569911e+01
25%      7.790189e+00
50%      1.667775e+01
75%      2.532557e+01
max      4.852972e+01
Name: AT_mean, dtype: float64

In [18]:
climate['year'].unique()

array([1960, 1961, 1962, 1963, 1964, 1965, 1966, 1967, 1968, 1969, 1970,
       1971, 1972, 1973, 1974, 1975, 1976, 1977, 1978, 1979, 1980, 1981,
       1982, 1983, 1984, 1985, 1986, 1987, 1988, 1989, 1990, 1991, 1992,
       1993, 1994, 1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003,
       2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014,
       2015, 2016, 2017, 2018, 2019])

In [19]:
climate = climate.sort_values(['NAME', 'year', 'month'])
climate

,NAME,fips,year,month,date,tMean,tMin,tMax,AT_mean,AT_min,AT_max
920430,Albany,36001,1960,1,1960-01-01,-7.164961,-13.460060,-0.869862,-7.104883,-13.205012,-0.838658
920431,Albany,36001,1960,1,1960-01-02,-9.151526,-17.303679,-0.999372,-9.044999,-16.871308,-0.969692
920432,Albany,36001,1960,1,1960-01-03,-0.437482,-6.560136,5.685172,-0.400462,-6.511033,5.941212
920433,Albany,36001,1960,1,1960-01-04,0.865364,-2.200579,3.931308,0.926935,-2.180386,4.096169
920434,Albany,36001,1960,1,1960-01-05,-3.921627,-8.507685,0.664432,-3.901230,-8.417892,0.721508
...,...,...,...,...,...,...,...,...,...,...,...
328720,Tulsa,40143,2019,12,2019-12-27,9.324806,3.188396,15.461216,9.856630,3.321969,16.781241
328721,Tulsa,40143,2019,12,2019-12-28,10.919987,6.090799,15.749176,11.614218,6.371573,17.117941
328722,Tulsa,40143,2019,12,2019-12-29,9.419642,4.413086,14.426197,9.960367,4.600517,15.580466
328723,Tulsa,40143,2019,12,2019-12-30,4.814748,0.652293,8.977202,5.022408,0.709106,9.477184


### find temperature data during 1981 to 2010

In [20]:
year_temp = [year for year in range(1981, 2011)]

In [21]:
temp = climate[climate['year'].isin(year_temp)]
temp

,NAME,fips,year,month,date,tMean,tMin,tMax,AT_mean,AT_min,AT_max
921081,Albany,36001,1981,1,1981-01-01,-13.010622,-17.088109,-8.933135,-12.773773,-16.666624,-8.832458
921082,Albany,36001,1981,1,1981-01-02,-10.055733,-14.977587,-5.133880,-9.923142,-14.656974,-5.104323
921083,Albany,36001,1981,1,1981-01-03,-13.646982,-20.944347,-6.349617,-13.384197,-20.313775,-6.303968
921084,Albany,36001,1981,1,1981-01-04,-20.129528,-27.156104,-13.102953,-19.545513,-26.137207,-12.862413
921085,Albany,36001,1981,1,1981-01-05,-18.218619,-23.987399,-12.449840,-17.738922,-23.173324,-12.234875
...,...,...,...,...,...,...,...,...,...,...,...
328441,Tulsa,40143,2010,12,2010-12-27,-1.708393,-8.535238,5.118452,-1.685310,-8.444760,5.342278
328442,Tulsa,40143,2010,12,2010-12-28,1.920808,-4.446112,8.287729,2.010414,-4.422628,8.728146
328443,Tulsa,40143,2010,12,2010-12-29,7.184651,2.119199,12.250103,7.539349,2.214927,13.101360
328444,Tulsa,40143,2010,12,2010-12-30,14.128174,9.294666,18.961681,15.237380,9.823680,20.957473


In [23]:
import datetime

temp['date'] = pd.to_datetime(temp['date'])
temp.head()

/tmp/ipykernel_1121010/108427414.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp['date'] = pd.to_datetime(temp['date'])


,NAME,fips,year,month,date,tMean,tMin,tMax,AT_mean,AT_min,AT_max
921081,Albany,36001,1981,1,1981-01-01,-13.010622,-17.088109,-8.933135,-12.773773,-16.666624,-8.832458
921082,Albany,36001,1981,1,1981-01-02,-10.055733,-14.977587,-5.133880,-9.923142,-14.656974,-5.104323
921083,Albany,36001,1981,1,1981-01-03,-13.646982,-20.944347,-6.349617,-13.384197,-20.313775,-6.303968
921084,Albany,36001,1981,1,1981-01-04,-20.129528,-27.156104,-13.102953,-19.545513,-26.137207,-12.862413
921085,Albany,36001,1981,1,1981-01-05,-18.218619,-23.987399,-12.449840,-17.738922,-23.173324,-12.234875


In [24]:
temp_valid = temp[(temp['month'] == 7) | (temp['month'] == 8)]
temp_valid

,NAME,fips,year,month,date,tMean,tMin,tMax,AT_mean,AT_min,AT_max
931956,Albany,36001,1981,7,1981-07-01,21.471490,13.813391,29.129589,24.074127,14.876260,34.363752
931957,Albany,36001,1981,7,1981-07-02,21.955519,17.752517,26.158522,24.688166,19.493703,30.216889
931958,Albany,36001,1981,7,1981-07-03,22.154882,16.649029,27.660734,24.942357,18.177728,32.286800
931959,Albany,36001,1981,7,1981-07-04,21.862821,17.744575,25.981068,24.570229,19.484164,29.975833
931960,Albany,36001,1981,7,1981-07-05,21.323068,17.491559,25.154576,23.886709,19.180822,28.862394
...,...,...,...,...,...,...,...,...,...,...,...
321121,Tulsa,40143,2010,8,2010-08-27,22.743002,13.694677,31.791328,25.696660,14.740402,38.272178
321122,Tulsa,40143,2010,8,2010-08-28,24.518575,15.593415,33.443736,28.015741,16.935671,40.800317
321123,Tulsa,40143,2010,8,2010-08-29,27.085653,20.183794,33.987512,31.488105,22.461366,41.650508
321124,Tulsa,40143,2010,8,2010-08-30,28.404342,22.384344,34.424341,33.331494,25.235863,42.340263


In [26]:
temp_valid['tMean'].isna().sum()

0

In [27]:
temp_valid['tMax'].isna().sum()

0

In [28]:
temp_valid['tMin'].isna().sum()

0

In [29]:
temp_valid = temp_valid.rename(columns = {'NAME': 'city_name'})

In [30]:
mean_temp_p85 = temp_valid.groupby(['city_name'])['AT_mean'].apply(lambda x: np.percentile(x.dropna(), 85)).reset_index()
mean_temp_p85.columns = ['city_name', 'AT_mean_p85']
mean_temp_p85

,city_name,AT_mean_p85
0,Albany,27.000275
1,Albuquerque,27.600961
2,Allentown,28.736612
3,Atlanta,32.423740
4,Austin,36.609987
5,Baltimore,32.905470
6,Baton Rouge,34.286830
7,Birmingham,33.209506
8,Boston,29.539582
9,Buffalo,26.705683


In [31]:
climate

,NAME,fips,year,month,date,tMean,tMin,tMax,AT_mean,AT_min,AT_max
920430,Albany,36001,1960,1,1960-01-01,-7.164961,-13.460060,-0.869862,-7.104883,-13.205012,-0.838658
920431,Albany,36001,1960,1,1960-01-02,-9.151526,-17.303679,-0.999372,-9.044999,-16.871308,-0.969692
920432,Albany,36001,1960,1,1960-01-03,-0.437482,-6.560136,5.685172,-0.400462,-6.511033,5.941212
920433,Albany,36001,1960,1,1960-01-04,0.865364,-2.200579,3.931308,0.926935,-2.180386,4.096169
920434,Albany,36001,1960,1,1960-01-05,-3.921627,-8.507685,0.664432,-3.901230,-8.417892,0.721508
...,...,...,...,...,...,...,...,...,...,...,...
328720,Tulsa,40143,2019,12,2019-12-27,9.324806,3.188396,15.461216,9.856630,3.321969,16.781241
328721,Tulsa,40143,2019,12,2019-12-28,10.919987,6.090799,15.749176,11.614218,6.371573,17.117941
328722,Tulsa,40143,2019,12,2019-12-29,9.419642,4.413086,14.426197,9.960367,4.600517,15.580466
328723,Tulsa,40143,2019,12,2019-12-30,4.814748,0.652293,8.977202,5.022408,0.709106,9.477184


### Find the heatwave Count

In [32]:
def findList(city_name, AT_mean_p85):
    temp_p85_list = []
    temp_p85_list.append(city_name)
    temp_p85_list.append(AT_mean_p85)
    return temp_p85_list

In [33]:
mean_temp_p85['temp_p85_list'] = mean_temp_p85.apply(lambda x: findList(x['city_name'], x['AT_mean_p85']), axis=1)
mean_temp_p85.head()

,city_name,AT_mean_p85,temp_p85_list
0,Albany,27.000275,"[Albany, 27.0002748482613]"
1,Albuquerque,27.600961,"[Albuquerque, 27.600960829292465]"
2,Allentown,28.736612,"[Allentown, 28.736612259322975]"
3,Atlanta,32.423740,"[Atlanta, 32.423739590069246]"
4,Austin,36.609987,"[Austin, 36.609986568434074]"


In [35]:
temp_p85_list = mean_temp_p85['temp_p85_list'].tolist()
temp_p85_list[:5]

[['Albany', 27.0002748482613],
 ['Albuquerque', 27.600960829292465],
 ['Allentown', 28.736612259322975],
 ['Atlanta', 32.423739590069246],
 ['Austin', 36.609986568434074]]

In [36]:
def findHeatwaveCount(temp, city, temp_p85):
    df = temp[temp['city_name'] == city]
    c1 = df['AT_min'].ge(temp_p85)
    N = 2
    g = (c1 != c1.shift()).cumsum()
    df1 = df.assign(
        cnt=df.groupby(g).date.transform('count')
      , n=df.groupby(g).agg('cumcount')
      , g=g
    )
    c2 = df1.cnt.ge(N)
    c3 = df1.n.mod(N).eq(0) & df1.n.le(df1.cnt-N)
    df['flag'] = np.where(c1 & c2 & c3, 1, 0)
    del(df1)
    return df

In [37]:
climate

,NAME,fips,year,month,date,tMean,tMin,tMax,AT_mean,AT_min,AT_max
920430,Albany,36001,1960,1,1960-01-01,-7.164961,-13.460060,-0.869862,-7.104883,-13.205012,-0.838658
920431,Albany,36001,1960,1,1960-01-02,-9.151526,-17.303679,-0.999372,-9.044999,-16.871308,-0.969692
920432,Albany,36001,1960,1,1960-01-03,-0.437482,-6.560136,5.685172,-0.400462,-6.511033,5.941212
920433,Albany,36001,1960,1,1960-01-04,0.865364,-2.200579,3.931308,0.926935,-2.180386,4.096169
920434,Albany,36001,1960,1,1960-01-05,-3.921627,-8.507685,0.664432,-3.901230,-8.417892,0.721508
...,...,...,...,...,...,...,...,...,...,...,...
328720,Tulsa,40143,2019,12,2019-12-27,9.324806,3.188396,15.461216,9.856630,3.321969,16.781241
328721,Tulsa,40143,2019,12,2019-12-28,10.919987,6.090799,15.749176,11.614218,6.371573,17.117941
328722,Tulsa,40143,2019,12,2019-12-29,9.419642,4.413086,14.426197,9.960367,4.600517,15.580466
328723,Tulsa,40143,2019,12,2019-12-30,4.814748,0.652293,8.977202,5.022408,0.709106,9.477184


In [39]:
climate = climate.rename(columns = {'NAME': 'city_name'})

In [40]:
df = climate.merge(mean_temp_p85, on='city_name', how='left')

In [41]:
df.head()

,city_name,fips,year,month,date,tMean,tMin,tMax,AT_mean,AT_min,AT_max,AT_mean_p85,temp_p85_list
0,Albany,36001,1960,1,1960-01-01,-7.164961,-13.460060,-0.869862,-7.104883,-13.205012,-0.838658,27.000275,"[Albany, 27.0002748482613]"
1,Albany,36001,1960,1,1960-01-02,-9.151526,-17.303679,-0.999372,-9.044999,-16.871308,-0.969692,27.000275,"[Albany, 27.0002748482613]"
2,Albany,36001,1960,1,1960-01-03,-0.437482,-6.560136,5.685172,-0.400462,-6.511033,5.941212,27.000275,"[Albany, 27.0002748482613]"
3,Albany,36001,1960,1,1960-01-04,0.865364,-2.200579,3.931308,0.926935,-2.180386,4.096169,27.000275,"[Albany, 27.0002748482613]"
4,Albany,36001,1960,1,1960-01-05,-3.921627,-8.507685,0.664432,-3.901230,-8.417892,0.721508,27.000275,"[Albany, 27.0002748482613]"


In [42]:
# Function to count heatwaves
def count_heatwaves(series, threshold):
    # Create a boolean series where True represents exceeding the threshold
    is_above_threshold = series > threshold

    # Identify groups of consecutive days above threshold
    # The diff and cumsum methods identify groups of consecutive True values
    heatwave_groups = (is_above_threshold.diff(1) != 0).cumsum()

    # Count the length of each group
    group_lengths = heatwave_groups[is_above_threshold].value_counts()

    # Count the groups that are at least two days long
    heatwave_count = (group_lengths >= 2).sum()

    return heatwave_count

In [43]:
# Apply the function to each year and fips
heatwave_counts = df.groupby(['year', 'month', 'city_name']).apply(lambda group: count_heatwaves(group['AT_min'], group['AT_mean_p85'].iloc[0]))

# Reset the index
heatwave_counts = heatwave_counts.reset_index(name='heatwave_count')
heatwave_counts

,year,month,city_name,heatwave_count
0,1960,1,Albany,0
1,1960,1,Albuquerque,0
2,1960,1,Allentown,0
3,1960,1,Atlanta,0
4,1960,1,Austin,0
...,...,...,...,...
35995,2019,12,Seattle,0
35996,2019,12,St. Louis,0
35997,2019,12,Tampa,0
35998,2019,12,Tucson,0


In [44]:
heatwave_counts['heatwave_count'].describe()

count    36000.0
mean         0.0
std          0.0
min          0.0
25%          0.0
50%          0.0
75%          0.0
max          0.0
Name: heatwave_count, dtype: float64

In [ ]:
heatwave_counts.to_csv('city_heatwave/city_heatwave_count_monthly_1960_2019.csv')

In [51]:
heatwave_counts['heatwave_count'].unique()

array([0, 3, 1, 4, 2, 5, 7, 6])

In [52]:
import pandas as pd
import numpy as np

# Assuming df is your DataFrame and has columns 'city', 'date', 'temperature'
# Assuming percentile_dict is your dictionary with 85th percentile temperatures for each city

# Sort by city and date
df = df.sort_values(['city_name', 'month', 'date'])

In [53]:
years = [year for year in range(1961, 2022)]

In [54]:
# Define a function to calculate heatwave durations for each city
def calculate_heatwave(df, years):
    pattern = []
    for year in years:
        # Create a boolean series where True indicates a day is part of a heatwave
        city_df = df[df['year'] == year] 
        for city in city_df['city_name'].unique():
            df_city = city_df[city_df['city_name'] == city]
            percentile = df_city['mean_temp_p85']
            is_heatwave = df_city['AT_min'] > percentile

            # Identify where True values change to False and vice versa,
            # and assign each group of consecutive True values a unique identifier
            heatwave_id = (is_heatwave.diff() != 0).cumsum()

            # Group by the identifiers and count the size of each group
            heatwave_lengths = heatwave_id[is_heatwave].value_counts()

            # Filter out heatwaves that are less than 2 days long
            heatwave_lengths = heatwave_lengths[heatwave_lengths >= 2]

            # Create a DataFrame with the start date of each heatwave and its duration
            heatwaves = pd.DataFrame({
                'city_name': city,
                'duration': heatwave_lengths.values,
                'year': year
            })
            pattern.append(heatwaves)
    
    return pattern

In [55]:
pattern = calculate_heatwave(df, years)
heatwave_durations = pd.concat(pattern)
heatwave_durations

,city_name,duration,year
0,Buffalo,3,1961
0,Hartford,2,1961
0,Jacksonville,2,1961
1,Jacksonville,2,1961
2,Jacksonville,2,1961
...,...,...,...
3,Tampa,3,2021
4,Tampa,3,2021
5,Tampa,3,2021
6,Tampa,2,2021


In [56]:
# Function to calculate heatwave durations
def calculate_heatwave_durations(series, threshold):
    # Create a boolean series where True represents exceeding the threshold
    is_above_threshold = series > threshold

    # Identify groups of consecutive days above threshold
    # The diff and cumsum methods identify groups of consecutive True values
    heatwave_groups = (is_above_threshold.diff(1) != 0).cumsum()

    # Count the length of each group
    group_lengths = heatwave_groups[is_above_threshold].value_counts()

    # Filter for groups that are at least two days long, and return their lengths
    heatwave_durations = group_lengths[group_lengths >= 2]

    return heatwave_durations

In [57]:
# Apply the function to each year and fips
heatwave_durations = df.groupby(['year', 'city_name']).apply(lambda group: calculate_heatwave_durations(group['AT_min'], group['mean_temp_p85'].iloc[0]))
# This will result in a series where each element is a series of heat wave durations
# To convert this into a DataFrame where each row represents a heat wave, you can do:
heatwave_durations = heatwave_durations.explode().reset_index(name='heatwave_duration')
heatwave_durations

,year,city_name,AT_min,heatwave_duration
0,1961,Buffalo,6,3
1,1961,Hartford,2,2
2,1961,Jacksonville,30,2
3,1961,Jacksonville,48,2
4,1961,Jacksonville,12,2
...,...,...,...,...
2386,2021,Tampa,4,3
2387,2021,Tampa,6,3
2388,2021,Tampa,20,3
2389,2021,Tampa,12,2


In [58]:
heatwave_durations.to_csv('city_heatwave/city_heatwave_duration_1961_2021.csv')

In [59]:
df['date'] = pd.to_datetime(df['date'])

In [60]:
# Function to calculate heatwave dates
def calculate_heatwave_dates(series, threshold):
    # Create a boolean series where True represents exceeding the threshold
    is_above_threshold = series['AT_min'] > threshold

    # Identify groups of consecutive days above threshold
    # The diff and cumsum methods identify groups of consecutive True values
    heatwave_groups = (is_above_threshold.diff(1) != 0).cumsum()
    
    # Get the dates of each group
    group_dates = series.loc[is_above_threshold, 'date'].groupby(heatwave_groups)

    # Filter for groups that are at least two days long, and return their dates
    heatwave_dates = group_dates.filter(lambda dates: len(dates) >= 2)

    return heatwave_dates

In [61]:
# Apply the function to each year and fips
heatwave_dates = df.groupby(['year', 'city_name']).apply(lambda group: calculate_heatwave_dates(group[['date', 'AT_min']], group['mean_temp_p85'].iloc[0]))
heatwave_dates

year  city_name         
1961  Buffalo    219650    1961-09-02
                 219651    1961-09-03
                 219652    1961-09-04
      Hartford   523936    1961-07-24
                 523937    1961-07-25
                              ...    
2021  Tampa      1199405   2021-08-21
                 1199406   2021-08-22
                 1199407   2021-08-23
                 1199470   2021-10-25
                 1199471   2021-10-26
Name: date, Length: 8394, dtype: datetime64[ns]

In [62]:
# The season length for each year and fips is the difference between the max and min date
season_lengths = heatwave_dates.groupby(level=['year','city_name']).apply(lambda dates: dates.max() - dates.min())
season_lengths = season_lengths.explode().reset_index(name='heatwave_season_length')
season_lengths

,year,city_name,heatwave_season_length
0,1961,Buffalo,2 days
1,1961,Hartford,1 days
2,1961,Jacksonville,43 days
3,1961,Miami,87 days
4,1961,New Orleans,1 days
...,...,...,...
885,2021,Portland,46 days
886,2021,Providence,61 days
887,2021,Rochester,1 days
888,2021,Seattle,1 days


In [63]:
def findDaysNumber(days):
    days = str(days)
    return int(days.split()[0])+1

In [64]:
season_lengths['heatwave_length'] = season_lengths['heatwave_season_length'].apply(findDaysNumber)
season_lengths

,year,city_name,heatwave_season_length,heatwave_length
0,1961,Buffalo,2 days,3
1,1961,Hartford,1 days,2
2,1961,Jacksonville,43 days,44
3,1961,Miami,87 days,88
4,1961,New Orleans,1 days,2
...,...,...,...,...
885,2021,Portland,46 days,47
886,2021,Providence,61 days,62
887,2021,Rochester,1 days,2
888,2021,Seattle,1 days,2


In [65]:
season_lengths = season_lengths.drop('heatwave_season_length', axis = 1)
season_lengths = season_lengths.rename(columns = {'heatwave_length': 'heatwave_season_length'})
season_lengths

,year,city_name,heatwave_season_length
0,1961,Buffalo,3
1,1961,Hartford,2
2,1961,Jacksonville,44
3,1961,Miami,88
4,1961,New Orleans,2
...,...,...,...
885,2021,Portland,47
886,2021,Providence,62
887,2021,Rochester,2
888,2021,Seattle,2


In [67]:
season_lengths.to_csv('city_heatwave/city_heatwave_season_length_1961_2021.csv')

In [68]:
def celsius_to_fahrenheit(c):
    f = c * 9/5 + 32
    return f

In [69]:
df['AT_min'] = df['AT_min'].apply(celsius_to_fahrenheit)
df['mean_temp_p85'] = df['mean_temp_p85'].apply(celsius_to_fahrenheit)

In [70]:
# Function to calculate average exceeded temperatures above threshold during heatwaves
def calculate_heatwave_exceeded_temp_avgs(series, threshold):
    # Create a boolean series where True represents exceeding the threshold
    is_above_threshold = series['AT_min'] > threshold

    # Identify groups of consecutive days above threshold
    # The diff and cumsum methods identify groups of consecutive True values
    heatwave_groups = (is_above_threshold.diff(1) != 0).cumsum()

    # Get the mean of exceeded temperatures (temperature - threshold) above threshold in each group
    group_exceeded_temp_avgs = (series.loc[is_above_threshold, 'AT_min'] - threshold).groupby(heatwave_groups).mean()

    # Filter for groups that are at least two days long, and return their exceeded temperature averages
    heatwave_exceeded_temp_avgs = group_exceeded_temp_avgs[group_exceeded_temp_avgs.index.map(is_above_threshold.groupby(heatwave_groups).size() >= 2)]

    return heatwave_exceeded_temp_avgs

In [72]:
# Apply the function to each year and fips
heatwave_exceeded_temp_avgs = df.groupby(['year', 'city_name']).apply(lambda group: calculate_heatwave_exceeded_temp_avgs(group[['AT_min']], group['mean_temp_p85'].iloc[0]))
heatwave_exceeded_temp_avgs 

year  city_name     AT_min
1961  Buffalo       6         4.089491
      Hartford      2         1.827083
      Jacksonville  12        0.956004
                    30        0.956004
                    48        0.956004
                                ...   
2021  Tampa         12        3.034205
                    14        3.660569
                    18        1.802730
                    20        1.802730
                    32        1.802730
Name: AT_min, Length: 2391, dtype: float64

In [73]:
# This will result in a series where each element is a series of heat wave exceeded temperature averages
# To convert this into a DataFrame where each row represents a heat wave, you can do:
heatwave_exceeded_temp_avgs = heatwave_exceeded_temp_avgs.explode().reset_index(name='heatwave_exceeded_temp_avg')
heatwave_exceeded_temp_avgs

,year,city_name,AT_min,heatwave_exceeded_temp_avg
0,1961,Buffalo,6,4.089491
1,1961,Hartford,2,1.827083
2,1961,Jacksonville,12,0.956004
3,1961,Jacksonville,30,0.956004
4,1961,Jacksonville,48,0.956004
...,...,...,...,...
2386,2021,Tampa,12,3.034205
2387,2021,Tampa,14,3.660569
2388,2021,Tampa,18,1.802730
2389,2021,Tampa,20,1.802730


In [74]:
heatwave_exceeded_temp_avgs = heatwave_exceeded_temp_avgs.drop('AT_min', axis = 1)
heatwave_exceeded_temp_avgs = heatwave_exceeded_temp_avgs.rename(columns = {'heatwave_exceeded_temp_avg': 'heatwave_intensity'})
heatwave_exceeded_temp_avgs

,year,city_name,heatwave_intensity
0,1961,Buffalo,4.089491
1,1961,Hartford,1.827083
2,1961,Jacksonville,0.956004
3,1961,Jacksonville,0.956004
4,1961,Jacksonville,0.956004
...,...,...,...
2386,2021,Tampa,3.034205
2387,2021,Tampa,3.660569
2388,2021,Tampa,1.802730
2389,2021,Tampa,1.802730


In [75]:
heatwave_exceeded_temp_avgs.to_csv('city_heatwave/city_heatwave_intensity_1961_2021.csv')